In [1]:
import numpy as np
import scipy as sp
#import gridData as gd
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
#import pytraj as pt
import py3Dmol as pmol

import sys
import os
import tqdm
import gc
import copy

from collections import defaultdict
import itertools


import plotly as ply
ply.io.renderers.default="notebook"

import ipywidgets as widgets
from ipywidgets import interact

import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.cluster import OPTICS, cluster_optics_dbscan

## Testing py3Dmol for molecule viewing

In [ ]:
p = pmol.view(query='pdb:1ycr')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.show()

## testing plotly for interactive 3D rendering

In [ ]:
#Toy test system to test rendering of 3d scatter plots with plotly
pointsList=np.array([
    np.array([0.,0.,10.])+np.random.random(3),
    np.array([0.,0.,-10.])+np.random.random(3),
    np.array([1.,0.,0.])*5.+np.random.random(3),
    np.array([np.cos(120.*np.pi/180.),np.sin(120.*np.pi/180.),0])*5.+np.random.random(3),
    np.array([np.cos(-120*np.pi/180.),np.sin(-120.*np.pi/180.),0])*5.+np.random.random(3)
])
pointsFrame=pd.DataFrame(
    {'X':pointsList[:,0],
     'Y':pointsList[:,1],
     'Z':pointsList[:,2]})
pointsFrame

In [ ]:
import plotly as ply

In [ ]:
ply.io.renderers.default="notebook"

In [ ]:
go=ply.graph_objs
fig=go.Figure(data=[go.Scatter3d(x=pointsFrame['X'],y=pointsFrame['Y'],z=pointsFrame['Z'],
                                mode='markers')])
fig.show()

# Load Coordinate data sets

In [ ]:
dataDir="simulationData"

potFilePattern="pot_"
potDataFiles=[fileName for fileName in os.listdir(dataDir) \
              if potFilePattern in fileName]
print("potassium data file list:",potDataFiles)

print()
alphaFilePattern="atomCA_"
alphaDataFiles=[fileName for fileName in os.listdir(dataDir) \
                if alphaFilePattern in fileName] 
print("alpha carbon control atom data file list:",alphaDataFiles)

In [ ]:
atomDataFrames=[]
for potFile in tqdm.tqdm_notebook(potDataFiles,desc='Loading Potassium Data'):
    potID=potFile.split('.')[0].split('_')[-1]
    potFilePath='/'.join([dataDir,potFile])
    tempFrame=pd.read_csv(potFilePath,delim_whitespace=True,skiprows=1,
                          names=['Frame','X','Y','Z','Unused1','Unused2','Unused3'])
    tempFrame=tempFrame.drop(columns=[colName for colName in tempFrame.columns \
                                      if 'Unused' in colName])
    tempFrame['AtomID']=potID
    tempFrame['AtomType']='POT'
    tempFrame=tempFrame[['AtomID','AtomType','Frame','X','Y','Z']]
    atomDataFrames.append(tempFrame.copy())
    
for alphaFile in tqdm.tqdm_notebook(alphaDataFiles,'Loading Alpha Carbon Data'):
    alphaID=alphaFile.split('.')[0].split('_')[-1].replace('resid','')
    alphaFilePath='/'.join([dataDir,alphaFile])
    tempFrame=pd.read_csv(alphaFilePath,delim_whitespace=True,skiprows=1,
                          names=['Frame','X','Y','Z','Unused1','Unused2','Unused3'])
    tempFrame=tempFrame.drop(columns=[colName for colName in tempFrame.columns \
                                      if 'Unused' in colName])
    tempFrame['AtomID']=alphaID
    tempFrame['AtomType']='CA'
    tempFrame=tempFrame[['AtomID','AtomType','Frame','X','Y','Z']]
    atomDataFrames.append(tempFrame.copy())
    
atomDataFrame=pd.concat(atomDataFrames)
atomDataFrames=[]
del(atomDataFrames)
atomDataFrame.head()

In [ ]:
atomDataFrame.AtomType.unique()

In [ ]:
atomDataFrame.to_csv('/'.join([dataDir,'atom_coordinate_data_table.csv']),
                     index=False)

## Visualize Loaded Atom Data as 3D scatter plot

In [ ]:
atomDataFrame=pd.read_csv('/'.join([dataDir,'atom_coordinate_data_table.csv']))
atomDataFrame.head()

In [ ]:
go=ply.graph_objs
fig=go.Figure(
    data=[
            go.Scatter3d(x=atomDataFrame.query('AtomType == "POT"')['X'],
                         y=atomDataFrame.query('AtomType == "POT"')['Y'],
                         z=atomDataFrame.query('AtomType == "POT"')['Z'],
                         mode='markers',
                         marker=dict(size=2,
                                     color='blue',
                                     colorscale='Viridis',
                                     opacity=.125)),
            go.Scatter3d(x=atomDataFrame.query('AtomType == "CA"')['X'],
                         y=atomDataFrame.query('AtomType == "CA"')['Y'],
                         z=atomDataFrame.query('AtomType == "CA"')['Z'],
                         mode='markers',
                         marker=dict(size=12,color='red',opacity=.75))
         ],
    layout=dict(width=800,height=600)
)
fig.show()

# Compute Potassium to Alpha Carbon Distances

In [ ]:
tempFrames=[]
baseFrame=atomDataFrame.set_index('Frame')
for groupName,groupData in \
    tqdm.tqdm_notebook(atomDataFrame.query('AtomType == "CA"').groupby('AtomID'),
                       desc='Computing Distances'):
    tempFrame=baseFrame.join(groupData.set_index('Frame')[['X','Y','Z']],
                             rsuffix='_ref',how='outer')
    tempFrame['RefAtomID']=groupName
    tempFrame['Distance']=tempFrame[['X','Y','Z','X_ref','Y_ref','Z_ref']].apply(
        lambda x: np.sqrt(np.sum(
            [(x[0]-x[3])**2,
             (x[1]-x[4])**2,
             (x[2]-x[5])**2])),
        axis=1)
    tempFrames.append(tempFrame.copy())

diffData=pd.concat(tempFrames)
tempFrames=[]
del(tempFrames)
gc.collect()
diffData.head()

In [ ]:
diffCoords=diffData.drop(
        columns=[colName for colName in diffData.columns if 'ref' in colName]
    ).reset_index().pivot_table(index=['Frame','AtomID','AtomType','X','Y','Z'],
                                columns='RefAtomID',values='Distance')
diffCoords.columns=np.array(diffCoords.columns.map(lambda x: 'D_%g'%(x)))
diffCoords=diffCoords.reset_index()
diffCoords.head()

In [ ]:
diffCoords.to_csv('/'.join([dataDir,'potassium_distance_data.csv']),
                  index=False)

# Visualize Potassium to Alpha Carbon Distances

We will use an ipython widget to allow the user to select which alpha carbon to use for colormapping the
potassium to alpha carbon distances.

In [ ]:
diffCoords=pd.read_csv('/'.join([dataDir,'potassium_distance_data.csv']))
diffCoords.head()

In [ ]:
diffCoords.sort_values(['AtomID','Frame']).tail()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
@interact
def color_by_distance(distanceColumn=[colName for colName in diffCoords.columns \
                                      if 'D_' in colName]):
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=diffCoords.query('AtomType == "POT"')['X'],
                             y=diffCoords.query('AtomType == "POT"')['Y'],
                             z=diffCoords.query('AtomType == "POT"')['Z'],
                             mode='markers',
                             marker=dict(size=2,
                                         color=diffCoords.query(
                                                 'AtomType == "POT"'
                                             )[distanceColumn],
                                         colorscale='RdBu',
                                         opacity=.125)),
                go.Scatter3d(
                    x=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['X'],
                    y=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['Y'],
                    z=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['Z'],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25)),
                go.Scatter3d(x=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['X'],
                             y=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['Y'],
                             z=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['Z'],
                             mode='markers',
                             marker=dict(size=12,color='red',opacity=.75))
             ],
        layout=dict(width=800,height=600)
    )
    fig.show()

# Compute PCA projection

In [ ]:
import sklearn as skl
from sklearn.decomposition import PCA

In [ ]:
Xdata=diffCoords.query('AtomType == "POT"')[diffCoords.columns[-5:]]
Xdata.head()

In [ ]:
pca=PCA()

pca.fit(Xdata)

pca_coords=pca.transform(diffCoords[diffCoords.columns[-5:]])
pca_coords[:10]

In [ ]:
pcaData=diffCoords.join(
    pd.DataFrame(pca_coords,
                 columns=['PCA_%g'%ii for ii in np.arange(pca_coords.shape[1])]))
pcaData.head()

In [ ]:
pcaData.to_csv('/'.join([dataDir,'pca_projection_data.csv']),
                  index=False)

# Visualize PCA projection data

Lets see how well the first 3 principal components do at reconstructing our data set.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact_manual

In [ ]:
pcaData=pd.read_csv('/'.join([dataDir,'pca_projection_data.csv']))
pcaData.head()

In [ ]:
@interact_manual
def color_by_distance(colorColumn=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='D_958'),
                      xCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_0'),
                      yCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_1'),
                      zCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_2'),
                     ):
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaData.query('AtomType == "POT"')[xCol],
                             y=pcaData.query('AtomType == "POT"')[yCol],
                             z=pcaData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color=pcaData.query(
                                                 'AtomType == "POT"'
                                             )[colorColumn],
                                         colorscale='RdBu',
                                         opacity=.125)),
                go.Scatter3d(
                    x=pcaData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

In [ ]:
@interact_manual
def testingFun( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    X_bound=widgets.FloatRangeSlider(     min=pcaData.X.min(),max=pcaData.X.max(),
                                   value=(pcaData.X.min(),pcaData.X.max())),
    
    Y_bound=widgets.FloatRangeSlider(     min=pcaData.Y.min(),max=pcaData.Y.max(),
                                   value=(pcaData.Y.min(),pcaData.Y.max())),
    
    Z_bound=widgets.FloatRangeSlider(     min=pcaData.Z.min(),max=pcaData.Z.max(),
                                   value=(pcaData.Z.min(),pcaData.Z.max())),
    
    D_958_bound=widgets.FloatRangeSlider( min=pcaData.D_958.min(),max=pcaData.D_958.max(),
                                   value=(pcaData.D_958.min(),pcaData.D_958.max())),
    
    D_1178_bound=widgets.FloatRangeSlider(min=pcaData.D_1178.min(),max=pcaData.D_1178.max(),
                                   value=(pcaData.D_1178.min(),pcaData.D_1178.max())),
    
    D_1366_bound=widgets.FloatRangeSlider(min=pcaData.D_1366.min(),max=pcaData.D_1366.max(),
                                   value=(pcaData.D_1366.min(),pcaData.D_1366.max())),
    
    D_2377_bound=widgets.FloatRangeSlider(min=pcaData.D_2377.min(),max=pcaData.D_2377.max(),
                                   value=(pcaData.D_2377.min(),pcaData.D_2377.max())),
    
    D_3796_bound=widgets.FloatRangeSlider(min=pcaData.D_3796.min(),max=pcaData.D_3796.max(),
                                   value=(pcaData.D_3796.min(),pcaData.D_3796.max())),
    
    PCA_0_bound=widgets.FloatRangeSlider( min=pcaData.PCA_0.min(),max=pcaData.PCA_0.max(),
                                   value=(pcaData.PCA_0.min(),pcaData.PCA_0.max())),
    
    PCA_1_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min(),max=pcaData.PCA_1.max(),
                                   value=(pcaData.PCA_1.min(),pcaData.PCA_1.max())),
    
    PCA_2_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min(),max=pcaData.PCA_1.max(),
                                   value=(pcaData.PCA_1.min(),pcaData.PCA_1.max())),
    
    PCA_3_bound=widgets.FloatRangeSlider( min=pcaData.PCA_3.min(),max=pcaData.PCA_3.max(),
                                   value=(pcaData.PCA_3.min(),pcaData.PCA_3.max())),
    
    PCA_4_bound=widgets.FloatRangeSlider( min=pcaData.PCA_4.min(),max=pcaData.PCA_4.max(),
                                   value=(pcaData.PCA_4.min(),pcaData.PCA_4.max())),
    xCol=widgets.Dropdown(options=pcaData.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=pcaData.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=pcaData.columns[3:],value='Z'),
    ):
    kwargDict=dict(locals())
    print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:13]
    for kwargKey in kwargKeys:
        print(kwargKey,end="")
        print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} > {factorMin:f}) and ({factorKey:s} < {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    print("query:",queryStr)
    print("selection shape:",pcaData.query(queryStr).shape)

In [ ]:
@interact_manual
def explore_milestone_bounds( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    X_bound=widgets.FloatRangeSlider(     min=pcaData.X.min()-.1,max=pcaData.X.max()+.1,
                                   value=(pcaData.X.min()-.1,pcaData.X.max()+.1)),
    
    Y_bound=widgets.FloatRangeSlider(     min=pcaData.Y.min()-.1,max=pcaData.Y.max()+.1,
                                   value=(pcaData.Y.min()-.1,pcaData.Y.max()+.1)),
    
    Z_bound=widgets.FloatRangeSlider(     min=pcaData.Z.min()-.1,max=pcaData.Z.max()+.1,
                                   value=(pcaData.Z.min()-.1,pcaData.Z.max()+.1)),
    
    D_958_bound=widgets.FloatRangeSlider( min=pcaData.D_958.min()-.1,max=pcaData.D_958.max()+.1,
                                   value=(pcaData.D_958.min()-.1,pcaData.D_958.max()+.1)),
    
    D_1178_bound=widgets.FloatRangeSlider(min=pcaData.D_1178.min()-.1,max=pcaData.D_1178.max()+.1,
                                   value=(pcaData.D_1178.min()-.1,pcaData.D_1178.max()+.1)),
    
    D_1366_bound=widgets.FloatRangeSlider(min=pcaData.D_1366.min()-.1,max=pcaData.D_1366.max()+.1,
                                   value=(pcaData.D_1366.min()-.1,pcaData.D_1366.max()+.1)),
    
    D_2377_bound=widgets.FloatRangeSlider(min=pcaData.D_2377.min()-.1,max=pcaData.D_2377.max()+.1,
                                   value=(pcaData.D_2377.min()-.1,pcaData.D_2377.max()+.1)),
    
    D_3796_bound=widgets.FloatRangeSlider(min=pcaData.D_3796.min()-.1,max=pcaData.D_3796.max()+.1,
                                   value=(pcaData.D_3796.min()-.1,pcaData.D_3796.max()+.1)),
    
    PCA_0_bound=widgets.FloatRangeSlider( min=pcaData.PCA_0.min()-.1,max=pcaData.PCA_0.max()+.1,
                                   value=(pcaData.PCA_0.min()-.1,pcaData.PCA_0.max()+.1)),
    
    PCA_1_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min()-.1,max=pcaData.PCA_1.max()+.1,
                                   value=(pcaData.PCA_1.min()-.1,pcaData.PCA_1.max()+.1)),
    
    PCA_2_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min()-.1,max=pcaData.PCA_1.max()+.1,
                                   value=(pcaData.PCA_1.min()-.1,pcaData.PCA_1.max()+.1)),
    
    PCA_3_bound=widgets.FloatRangeSlider( min=pcaData.PCA_3.min()-.1,max=pcaData.PCA_3.max()+.1,
                                   value=(pcaData.PCA_3.min()-.1,pcaData.PCA_3.max()+.1)),
    
    PCA_4_bound=widgets.FloatRangeSlider( min=pcaData.PCA_4.min(),max=pcaData.PCA_4.max(),
                                   value=(pcaData.PCA_4.min(),pcaData.PCA_4.max())),
    xCol=widgets.Dropdown(options=pcaData.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=pcaData.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=pcaData.columns[3:],value='Z'),
    verbose=widgets.ToggleButton(value=False)
    ):
    kwargDict=dict(locals())
    if verbose:
        print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:13]
    if verbose:
        for kwargKey in kwargKeys:
            print(kwargKey,end="")
            print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} >= {factorMin:f}) and ({factorKey:s} <= {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    if verbose:
        print("query:",queryStr)
        print("selection shape:",pcaData.query(queryStr).shape)
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaData.query('AtomType == "POT"')[xCol],
                             y=pcaData.query('AtomType == "POT"')[yCol],
                             z=pcaData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='grey',
                                         opacity=.0625)),
                go.Scatter3d(x=pcaData.query(queryStr)[xCol],
                             y=pcaData.query(queryStr)[yCol],
                             z=pcaData.query(queryStr)[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='blue',
                                         opacity=.25)),
                go.Scatter3d(
                    x=pcaData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

# Compute Cylindrical Coordinates

In [ ]:
import math
tqdm.tqdm.pandas(tqdm.tqdm_notebook)

In [ ]:
diffCoords.query('AtomType=="CA"').head()

In [ ]:
tqdm.tqdm.pandas(tqdm.tqdm_notebook)
cylCenters=diffCoords.query('AtomType=="CA"')[['Frame','X','Y','Z']].groupby(
    ['Frame']).progress_aggregate(np.mean)
diffCyl=diffCoords.set_index(['Frame']).join(
    cylCenters,rsuffix='_Center',how='outer')
diffCyl=diffCyl.reset_index()
diffCyl['R']=np.sqrt(
        (diffCyl['X']-diffCyl['X_Center'])**2 + \
        (diffCyl['Y']-diffCyl['Y_Center'])**2)
diffCyl['Theta']=np.arctan2(diffCyl['Y']-diffCyl['Y_Center'],diffCyl['X']-diffCyl['X_Center'])
diffCyl=diffCyl[np.concatenate([
    diffCyl.columns[:6],
    [colName for colName in diffCyl.columns if '_Center' in colName],
    ['R','Theta'],
    [colName for colName in diffCyl.columns if 'D_' in colName],
])]
print(diffCyl.shape)
diffCyl.head()

## Explore radial cutoff

In [ ]:
@interact_manual
def explore_milestone_bounds( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    R_bound=widgets.FloatRangeSlider(     min=0,max=diffCyl.R.max()+.1,
                                   value=(0,diffCyl.R.max()+.1)),
    
    Theta_bound=widgets.FloatRangeSlider(     min=-np.pi,max=np.pi,
                                   value=(-np.pi,np.pi)),
    
    Z_bound=widgets.FloatRangeSlider(     min=diffCyl.Z.min()-.1,max=diffCyl.Z.max()+.1,
                                   value=(diffCyl.Z.min()-.1,diffCyl.Z.max()+.1)),
    xCol=widgets.Dropdown(options=diffCyl.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=diffCyl.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=diffCyl.columns[3:],value='Z'),
    showCenter=widgets.ToggleButton(value=True),
    verbose=widgets.ToggleButton(value=False)
    ):
    kwargDict=dict(locals())
    if verbose:
        print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:3]
    if verbose:
        for kwargKey in kwargKeys:
            print(kwargKey,end="")
            print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} >= {factorMin:f}) and ({factorKey:s} <= {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    if verbose:
        print("query:",queryStr)
        print("selection shape:",diffCyl.query(queryStr).shape)
        print(diffCyl.query(queryStr).head())
    go=ply.graph_objs
    plotData=[
                go.Scatter3d(x=diffCyl.query(
                                    'AtomType == "POT"'
                                 ).query("not ("+queryStr+")")[xCol],
                             y=diffCyl.query(
                                     'AtomType == "POT"'
                                 ).query("not ("+queryStr+")")[yCol],
                             z=diffCyl.query(
                                     'AtomType == "POT"'
                                 ).query("not ("+queryStr+")")[zCol],
                             mode='markers',
                             marker=dict(size=24,
                                         color='grey',
                                         opacity=.0625)),
                go.Scatter3d(x=diffCyl.query(queryStr)[xCol],
                             y=diffCyl.query(queryStr)[yCol],
                             z=diffCyl.query(queryStr)[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='blue',
                                         opacity=.25)),
                go.Scatter3d(
                    x=diffCyl.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=diffCyl.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=diffCyl.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ]
    if showCenter:
        plotData.append(
                go.Scatter3d(x=diffCyl.query('AtomType == "CA"')['X_Center'],
                             y=diffCyl.query('AtomType == "CA"')['Y_Center'],
                             z=diffCyl.query('AtomType == "CA"')['Z_Center'],
                             mode='markers',
                             marker=dict(size=12,
                                         color='red',
                                         opacity=.5)))
    fig=go.Figure(
        data=plotData,
        layout=dict(
            width=800,height=600)
    )
    fig.show()

## Apply Cylindrical Filter

In [ ]:
queryEntry="(R >= 0.000000) and (R <= 171.000000) and (Theta >= -3.141593) and (Theta <= 3.141593) and (Z >= 0.000000) and (Z <= 145)"
cylData=diffCyl.query(queryEntry)
print(cylData.shape,diffCoords.shape)
display(diffCyl.query("not ("+queryEntry+")"))
cylData.head()

## Compute Filtered PCA projection

In [ ]:
import sklearn as skl
from sklearn.decomposition import PCA

In [ ]:
queryEntry="(R >= 0.000000) and (R <= 80.000000) and (Theta >= -3.141593) and (Theta <= 3.141593) and (Z >= 0.000000) and (Z <= 140)"
cylData=diffCyl.query(queryEntry)

XCylData=cylData.query('AtomType == "POT"')[cylData.columns[-5:]]

pcaCyl=PCA()
pcaCyl.fit(XCylData)
pcaCyl_coords=pcaCyl.transform(cylData[cylData.columns[-5:]])

pcaCylData=cylData.copy()
for ii in np.arange(pcaCyl_coords.shape[1]):
    pcaCylData['PCAcyl_%g'%ii]=pcaCyl_coords[:,ii]
    
pcaCylData.head()

In [ ]:
?np.savetxt

In [ ]:
np.savetxt(fname='/'.join([dataDir,'pca_components.cylindricalFilter.txt']),X=pcaCyl.components_)
pcaCylData.to_csv('/'.join([dataDir,'pca_projection_data.cylindricalFilter.csv']),
                  index=False)

# Visualize Cylindrical Filtered PCA projection

In [ ]:
dataDir='simulationData'
pcaCylData=pd.read_csv('/'.join([dataDir,'pca_projection_data.cylindricalFilter.csv']))
pcaComponents=np.loadtxt('/'.join([dataDir,'pca_components.cylindricalFilter.txt']))
display(pcaCylData.head())
display(pcaCylData[pcaCylData.columns[3:]].agg(['min','mean','max','std']).T)
display(pcaComponents)

In [ ]:
@interact_manual
def color_by_distance(colorColumn=widgets.Dropdown(
                          options=[colName for colName in pcaCylData.columns \
                                   if ('PCAcyl_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCAcyl_0'),
                      xCol=widgets.Dropdown(
                          options=[colName for colName in pcaCylData.columns \
                                   if ('PCAcyl_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='X'),
                      yCol=widgets.Dropdown(
                          options=[colName for colName in pcaCylData.columns \
                                   if ('PCAcyl_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='Y'),
                      zCol=widgets.Dropdown(
                          options=[colName for colName in pcaCylData.columns \
                                   if ('PCAcyl_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='Z'),
                     ):
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaCylData.query('AtomType == "POT"')[xCol],
                             y=pcaCylData.query('AtomType == "POT"')[yCol],
                             z=pcaCylData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color=pcaCylData.query(
                                                 'AtomType == "POT"'
                                             )[colorColumn],
                                         colorscale='RdBu',
                                         opacity=.125)),
                go.Scatter3d(
                    x=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

In [ ]:
@interact_manual
def explore_milestone_bounds( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    X_bound=widgets.FloatRangeSlider(     min=pcaCylData.X.min()-1.,max=pcaCylData.X.max()+1.,
                                   value=(pcaCylData.X.min()-1.,pcaCylData.X.max()+1.)),
    
    Y_bound=widgets.FloatRangeSlider(     min=pcaCylData.Y.min()-1.,max=pcaCylData.Y.max()+1.,
                                   value=(pcaCylData.Y.min()-1.,pcaCylData.Y.max()+1.)),
    
    Z_bound=widgets.FloatRangeSlider(     min=pcaCylData.Z.min()-1.,max=pcaCylData.Z.max()+1.,
                                   value=(pcaCylData.Z.min()-1.,pcaCylData.Z.max()+1.)),
    
    R_bound=widgets.FloatRangeSlider(     min=0,max=pcaCylData.R.max()+1.,
                                   value=(0,pcaCylData.R.max()+1.)),
    
    Theta_bound=widgets.FloatRangeSlider(     min=-np.pi,max=np.pi,
                                   value=(-np.pi,np.pi)),
    
    D_958_bound=widgets.FloatRangeSlider( min=pcaCylData.D_958.min()-1.,max=pcaCylData.D_958.max()+1.,
                                   value=(pcaCylData.D_958.min()-1.,pcaCylData.D_958.max()+1.)),
    
    D_1178_bound=widgets.FloatRangeSlider(min=pcaCylData.D_1178.min()-1.,max=pcaCylData.D_1178.max()+1.,
                                   value=(pcaCylData.D_1178.min()-1.,pcaCylData.D_1178.max()+1.)),
    
    D_1366_bound=widgets.FloatRangeSlider(min=pcaCylData.D_1366.min()-1.,max=pcaCylData.D_1366.max()+1.,
                                   value=(pcaCylData.D_1366.min()-1.,pcaCylData.D_1366.max()+1.)),
    
    D_2377_bound=widgets.FloatRangeSlider(min=pcaCylData.D_2377.min()-1.,max=pcaCylData.D_2377.max()+1.,
                                   value=(pcaCylData.D_2377.min()-1.,pcaCylData.D_2377.max()+1.)),
    
    D_3796_bound=widgets.FloatRangeSlider(min=pcaCylData.D_3796.min()-1.,max=pcaCylData.D_3796.max()+1.,
                                   value=(pcaCylData.D_3796.min()-1.,pcaCylData.D_3796.max()+1.)),
    
    PCAcyl_0_bound=widgets.FloatRangeSlider( min=pcaCylData.PCAcyl_0.min()-1.,max=pcaCylData.PCAcyl_0.max()+1.,
                                   value=(pcaCylData.PCAcyl_0.min()-1.,pcaCylData.PCAcyl_0.max()+1.)),
    
    PCAcyl_1_bound=widgets.FloatRangeSlider( min=pcaCylData.PCAcyl_1.min()-1.,max=pcaCylData.PCAcyl_1.max()+1.,
                                   value=(pcaCylData.PCAcyl_1.min()-1.,pcaCylData.PCAcyl_1.max()+1.)),
    
    PCAcyl_2_bound=widgets.FloatRangeSlider( min=pcaCylData.PCAcyl_1.min()-1.,max=pcaCylData.PCAcyl_1.max()+1.,
                                   value=(pcaCylData.PCAcyl_1.min()-1.,pcaCylData.PCAcyl_1.max()+1.)),
    
    PCAcyl_3_bound=widgets.FloatRangeSlider( min=pcaCylData.PCAcyl_3.min()-1.,max=pcaCylData.PCAcyl_3.max()+1.,
                                   value=(pcaCylData.PCAcyl_3.min()-1.,pcaCylData.PCAcyl_3.max()+1.)),
    
    PCAcyl_4_bound=widgets.FloatRangeSlider( min=pcaCylData.PCAcyl_4.min(),max=pcaCylData.PCAcyl_4.max(),
                                   value=(pcaCylData.PCAcyl_4.min(),pcaCylData.PCAcyl_4.max())),
    
    xCol=widgets.Dropdown(options=pcaCylData.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=pcaCylData.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=pcaCylData.columns[3:],value='Z'),
    verbose=widgets.ToggleButton(value=False),
    showCutRegions=widgets.ToggleButton(value=False),
    ):
    kwargDict=dict(locals())
    if verbose:
        print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:15]
    if verbose:
        for kwargKey in kwargKeys:
            print(kwargKey,end="")
            print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} >= {factorMin:f}) and ({factorKey:s} <= {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    if verbose:
        print("query:",queryStr)
        print("selection shape:",pcaCylData.query(queryStr).shape)
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaCylData.query('AtomType == "POT"')[xCol],
                             y=pcaCylData.query('AtomType == "POT"')[yCol],
                             z=pcaCylData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='grey',
                                         opacity=.0625)),
                go.Scatter3d(x=pcaCylData.query(queryStr)[xCol],
                             y=pcaCylData.query(queryStr)[yCol],
                             z=pcaCylData.query(queryStr)[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='blue',
                                         opacity=.25)),
                go.Scatter3d(
                    x=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaCylData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=900,height=675)
    )
    fig.show()
    if showCutRegions:
        aggCols=['_'.join(keyName.split('_')[:-1]) for keyName in kwargKeys]
        summaryData=pcaCylData.query(
                "AtomType == 'POT'"
            ).query(
                queryStr
            )[aggCols].agg(['min','max','mean'])
        summaryData=summaryData.T
        #print(summaryData.columns)
        summaryData['Interval']=summaryData['min'].map(str) + \
            ' - '+summaryData['max'].map(str)
        display(summaryData)

# Explore using clustering to find milestone centers

## Generate clusters using agglomerative clustering

In [ ]:
dataDir='simulationData'
pcaCylData=pd.read_csv('/'.join([dataDir,'pca_projection_data.cylindricalFilter.csv']))

In [ ]:
clust=skl.cluster.AgglomerativeClustering(n_clusters=256)

clustDat=pcaCylData.query('AtomType == "POT"')

Xvals=clustDat[
    #[colName for colName in pcaCylData if 'PCAcyl' in colName]
        #['PCAcyl_2','PCAcyl_3','PCAcyl_1']
        ['X','Y','Z']
    ]
dataInds=pcaCylData.query('AtomType == "POT"').index

clust.fit(Xvals)

xinds=np.arange(len(Xvals))

#reachability=clust.reachability_[clust.ordering_]
#labels=clust.labels_[clust.ordering_]
labels=clust.labels_
clustDat['Cluster']=labels

labelSeries=clustDat['Cluster']
labelSeries.to_csv('/'.join([dataDir,'Agglomerative_Clustering_Labels']))

## Load Agglomerative Clustering Results from Disk

In [ ]:
dataDir='simulationData'
pcaCylData=pd.read_csv('/'.join([dataDir,'pca_projection_data.cylindricalFilter.csv']))
#labelSeries=pd.Series.from_csv('/'.join([dataDir,'Agglomerative_Clustering_Labels']))
clustDat=pcaCylData.query('AtomType == "POT"')
clustDat['Cluster']=labelSeries
clustDat.to_csv('/'.join([dataDir,'cluster_data.cylindricalFilter.csv']))
clustDat.head()

## Visualize Clusters

In [ ]:
plotData=clustDat
plotData['Cluster']=labels
xCol='X'#'PCAcyl_2'
yCol='Y'#'PCAcyl_3'
zCol='Z'#'PCAcyl_1'
nLabels=len(np.unique(labels))
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    scatterPlots.append(
        go.Scatter3d(
                 x=groupData[xCol],
                 y=groupData[yCol],
                 z=groupData[zCol],
                 mode='markers',
                 marker=dict(size=2,
                             color=colorpalette.as_hex()[iGroup],
                             opacity=.25)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675))
fig.show()

## Compute Cluster Summary and Perform Tesselation / Triangulation

In [ ]:
clusterSummary=plotData.groupby('Cluster').agg('mean')
clusterSummary.head()

In [ ]:
clusterSummary.to_csv('/'.join([dataDir,'Agglomerative_clustering_centers.csv']))

In [ ]:
clusterSummary=pd.read_csv('/'.join([dataDir,'Agglomerative_clustering_centers.csv']))
clusterSummary.head()

In [ ]:
tessellationClusters=clusterSummary.index.to_numpy() #[0,1,2,3,4,8,10] 
centeroidColumns=np.concatenate([
    ['Cluster'],
    clusterSummary.columns[3:6],
    clusterSummary.columns[11:]])
tessellationCoordinateColumns=['PCAcyl_2','PCAcyl_3','PCAcyl_1'] #[colName for colName in clusterCenteroids.columns \
                               #if 'PCAcyl' in colName]


clusterCenteroids=clusterSummary
tessellationData=clusterCenteroids[
        clusterCenteroids['Cluster'].isin(tessellationClusters)
    ][tessellationCoordinateColumns]
clusterVoronoi=sp.spatial.Voronoi(tessellationData)
clusterDelaunay=sp.spatial.Delaunay(tessellationData)
print('cluster centeroids')
display(clusterCenteroids)
print('cluster vertices')
print(clusterDelaunay.vertices)
print()
print("cluster adjacency")
neiList=defaultdict(set)
for p in clusterDelaunay.vertices:
    for i,j in itertools.combinations(p,2):
        neiList[tessellationClusters[i]].add(tessellationClusters[j])
        neiList[tessellationClusters[j]].add(tessellationClusters[i])

for key in sorted(neiList.keys()):
    print("%d:%s" % (key,','.join([str(i) for i in neiList[key]])))

## Visualize triangulation of cluster centers

xCol='X'
yCol='Y'
zCol='Z'

plotData=clusterCenteroids
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    scatterPlots.append(
        go.Scatter3d(
                 x=groupData[xCol],
                 y=groupData[yCol],
                 z=groupData[zCol],
                 name='Center_%g'%kClass,
                 mode='markers',
                 marker=dict(size=8,
                             symbol='circle-open',
                             color=colorpalette.as_hex()[kClass],
                             opacity=.75)))
    
    neighborPoints=neiList[kClass]
    for nbPoint in neighborPoints:
        neighborCenters=plotData[plotData['Cluster'].isin([kClass,nbPoint])]
        scatterPlots.append(
            go.Scatter3d(
                x=neighborCenters[xCol],
                y=neighborCenters[yCol],
                z=neighborCenters[zCol],
                name='%g - %g'%(kClass,nbPoint),
                mode='lines',
                line=dict(color='black')
            )
        )

plotData=clustDat
#plotData['Cluster']=labels
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
#scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    scatterPlots.append(
        go.Scatter3d(
                 x=groupData[xCol],
                 y=groupData[yCol],
                 z=groupData[zCol],
                 mode='markers',
                 name='Cluster_%g'%kClass,
                 marker=dict(size=4,
                             color=colorpalette.as_hex()[kClass],
                             opacity=.125)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675,
            scene = dict(
                    xaxis_title=xCol,
                    yaxis_title=yCol,
                    zaxis_title=zCol)))

fig.show()

## Cell Boundary / Occupancy derivation

The boundary equations / restraints for each cell can be modeled based upon the triangulation. 
Specifically, for each pair of connected cells, we can construct an inclusion test by taking the midpoint
of the two cell centers then computing the normal vector, n, directed from the midpoint, m, between the two cell centers c1 and c2 to a given cell center (either c1 or c2) i.e. n1 = c1-m, n2 = c2-m. Then any point in space, r, we say that r is on the same side of the dividing plane as cell 1 if dot( (r-m), n1 )>0 or r is in on the same side of the dividing plane as cell 2 if dot( (r-m), n2 ) >0.

From here we can then compute cell occupancy conditions for each cell as the union over all the half plane inclusion conditions for that cell center to each connected cell.

So for an N dimensional triangulation, each half plane inclusion condition is defined by a set of 2N numbers, the first N define the coordinates of the centeroid, m, and the second N define the normal vector n pointing from m to the cell being considered.

So for a given central cell, i, and test cell, j, we can compute whether coordinate r is cloer to i than to j:
<font size=4> 
$$
    \delta_{ij} = [(r-c_{ij}) \cdot (R_i-c_{ij})>0] \tag{1}
$$
</font>
where $\delta_{ij}$ is equal to 1 if $r$ is closer to $i$ than to $j$ and 0 if not, $c_{ij}$ is the position vector of the midpoint between cell i and cell j, and $R_i$ is the position vector of cell i

Using this we can then test each point ion coordinate from our data set to determine which cells it belongs to (i.e. for which cells does it pass all half plane inclusion tests).
Importantly, this will allow us to make sure that no ion coordinates belong to more than one cell.

Then, we can plot our cell occupancy as above and color any ion coordinates that occupy multiply cells in red.

As an important note, the inclusion equation above can be expressed as a linear combination of triangulation space coordinates (Distance PCA modes) plus / minus a constant. And since the triangulation space coordinates, being distance PCA modes, are themselves linear combinations of our distances, the inclusion equations can be recast as intervals over LCOD restraints using our defined distances!

To begin, we first note that we will need to recast our above expression in terms of an inequality over some linear combination of distance coordinates, while at present we have an inequality involving the dotproduct of vectors in PCA space, the bases of which are linear combinations of the distance coordinates.

To accomplish this derivation, we first convert from vector dot product notation into an explicit summation over PCA modes, then recast into LCOD notation recasting each PCA mode as a sum over distance coordinates.

To get $\delta_{ij}$ as an explicit sum over every PCA mode, $k$, we write:

<font size=4>
$$
\begin{align}
\delta_{ij} & = [((r-c_{ij}) \cdot (R_i-c_{ij})) & > 0] \nonumber \\
& = [(\sum\limits_{k \in PCAmodes}{r_k R_{ik}-r_k c_{ijk}-c_{ijk} p_{ik}+c_{ijk}^2 }) & > 0] \nonumber \\
& = [(\sum\limits_{k \in PCAmodes}{ r_k (R_{ik} - c_{ijk}) - c_{ijk} R_{ijk} + c_{ijk}^2 }) & > 0] \tag{2}  
\end{align} 
$$
</font>

We now need to convert $r_k$ from the PCA space coordinates into its equivalent under our distance based coordinates, $r_D$, which is what will be used for our restraints. To do this, we use the fact that each principal component mode $k$ is itself a linear combination (i.e. weighted sum) over all distance coordinates $D$. I.e.:

<font size=4> 
$$
    r_k = \sum\limits_{D \in Distances}{\alpha_{kD} (r_D - \hat{\lambda_D})} \tag{3}
$$
</font>
where $\alpha_{kD}$ is the component (coefficient) for distance $D$ under PCA mode $k$, $r_D$ is the position measured in terms of our defined distance measurements as discussed above, and $\hat{\lambda_D}$ is the mean of the measured distance coordinate from PCA.
We now replace $r_k$ in equation <b>(2)</b> with the above expression:

<font size=2>
$$
\begin{align}
\delta_{ij} & = [(\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances}{ \alpha_{kD}(r_{D} - \hat{\lambda_D}))(R_{ik} - c_{ijk}) - c_{ijk} R_{ijk} + c_{ijk}^2 }})) & > 0] \nonumber \\
& = [({\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances} \alpha_{kD}(R_{ik} - c_{ijk})(r_{D}-\hat{\lambda_D}))} - \sum\limits_{k \in PCAmodes}{c_{ijk} R_{ijk} + c_{ijk}^2 }})) & > 0] \nonumber \\
& = [({\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}-\alpha_{kD}(R_{ik} - c_{ijk})\hat{\lambda_D})} - \sum\limits_{k \in PCAmodes}{c_{ijk} R_{ijk} + c_{ijk}^2 }})) & > 0] \nonumber \\
& = [({\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances}{\alpha_{kD}(R_{ik} - c_{ijk})r_{D})}}) - (\sum\limits_{k \in PCAmodes}{c_{ijk} R_{ijk} + c_{ijk}^2 + (\sum\limits_{D \in Distances}{ \alpha_{kD}(R_{ik} - c_{ijk})\hat{\lambda_D}}}}))) & > 0] \nonumber \\
& = [({\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances}{\alpha_{kD}(R_{ik} - c_{ijk})r_{D})}}) - (\sum\limits_{k \in PCAmodes}{(c_{ijk} R_{ijk} + c_{ijk}^2 + (R_{ik} - c_{ijk})(\sum\limits_{D \in Distances}{ \alpha_{kD}\hat{\lambda_D}}})}))) & > 0] \tag{4} 
\end{align} 
$$
</font>

Where $\alpha_{kD}$ is the coefficient for the $D$'th distance in the $k$'th principal component mode.

Notice that the last equation <b>(4)</b> was deliberately rearranged on the second line to emphasize that the summation only affects the $r_D$ term and the coefficient of that term is now clearly recognizable as $\alpha_{kD}(R_{ik} - c_{ijk})$.

We can now derive our LCOD restraint by decomposing the right hand side expression of equation <B>(4)</B> into a LCOD coefficient term and an interval bound term. Lets rearrange this a bit to make our interval condition more clear. We will rearrange our inequality so that the term containing D is isolated on the left hand side.

<font size=3>
$$
\delta_{ij} = [(\sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}) } ) >  \sum\limits_{k \in PCAmodes}{(c_{ijk} R_{ijk} + c_{ijk}^2 + (R_{ik} - c_{ijk})(\sum\limits_{D \in Distances}{ \alpha_{kD}\hat{\lambda_D}}))} ] \tag{5} 
$$
</font>

We now have our LCOD collective variable and its corresponding interval limit clearly isolated. The expression we will use to derive our LCOD collective variable is on the left hand side of the inequality, while the expression that will be used to derive the interval over this collective variable is on the right hand side of the inequality.

Next we need to derive the individual LCOD coefficients, $\kappa_{ijD}$ for distance D so that we may clearly define our LCOD restraint which will be used to restrain our ion to be closer to cell $i$ than cell $j$. We do so by recognize that our LCOD restraint collective variable should take the form:
<font size=4>
$$
\begin{align}
    LCOD_{ij} & = \sum\limits_{k \in PCAmodes}{(\sum\limits_{D \in Distances} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}) } \nonumber \\
    LCOD_{ij} & = \sum\limits_{D \in Distances}{(\sum\limits_{k \in PCAmodes} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}) } \tag{6}
\end{align}
$$
</font>

Our $\kappa_{ijd}$ coefficients are then readily visibile as the terms being summed by the left hand side of the inequallity shown in the last line of equation <b>(4)</b> after swapping the positions of the PCA and distance coordinate summations:

<font size=4>
$$
\begin{align}
    LCOD_{ij} & = \sum\limits_{D \in Distances}{(\sum\limits_{k \in PCAmodes} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}) } \nonumber \\
    & = \sum\limits_{D \in Distances}{\kappa_{ijd}r_D};  \nonumber \\
    where & : \kappa_{ijD} = \sum\limits_{k \in PCAmodes}{\alpha_{kD} (R_{ik} - c_{ijk})} \tag{7}
\end{align}
$$
</font>

and the flat bottom region interval for this LCOD restraint, $LCOD_{ij}$ used to restrain our ion to be closer to cell $i$ than cell $j$ is given as:

<font size=4>
\begin{align}
    LCOD_{ij} > \sum\limits_{k \in PCAmodes}{(c_{ijk} R_{iK} - c_{ijk}^2 + (R_{ik} - c_{ijk})(\sum\limits_{D \in Distances}{ \alpha_{kD}\hat{\lambda_D}}) )} \tag{8} \\
\end{align}
</font>

In [ ]:
cTest=[15, 22, 30, 31, 54, 59, 63, 82, 100, 124, 135, 137, 141, 144,
       145, 146, 177, 186, 194, 202, 207, 211, 214, 215, 220, 223, 230]
plotClusters=cTest
xCol='PCAcyl_2' #'X' #
yCol='PCAcyl_3' #'Y' #
zCol='PCAcyl_1' #'Z' #

plotData=clusterCenteroids
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     name='Center_%g'%kClass,
                     mode='markers',
                     marker=dict(size=12,
                                 symbol='circle-open',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.75)))

        neighborPoints=neiList[kClass]
        for nbPoint in neighborPoints:
            if nbPoint in plotClusters:
                neighborCenters=plotData[plotData['Cluster'].isin([kClass,nbPoint])]
                scatterPlots.append(
                    go.Scatter3d(
                        x=neighborCenters[xCol],
                        y=neighborCenters[yCol],
                        z=neighborCenters[zCol],
                        name='%g - %g'%(kClass,nbPoint),
                        mode='lines',
                        line=dict(color='black')
                    )
                )

plotData=clustDat
#plotData['Cluster']=labels
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
#scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=6,
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.25)))
    else:
        kClass,groupData=plotGroup
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=2,
                                 color='black',
                                 opacity=.0625)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675,
            scene = dict(
                    xaxis_title=xCol,
                    yaxis_title=
           yCol,         zaxis_title=zCol)))

fig.show()

In [ ]:
cTest=[202,135,211,186,63]

plotClusters=cTest
xCol='X' #'PCAcyl_2' #
yCol='Y' #'PCAcyl_3' #
zCol='Z' #'PCAcyl_1' #

plotData=clusterCenteroids
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     name='Center_%g'%kClass,
                     mode='markers',
                     marker=dict(size=12,
                                 symbol='circle-open',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.75)))

        

        neighborPoints=neiList[kClass]
        for nbPoint in neighborPoints:
            if nbPoint in plotClusters:
                neighborCenters=plotData[plotData['Cluster'].isin([kClass,nbPoint])]
                scatterPlots.append(
                    go.Scatter3d(
                        x=neighborCenters[xCol],
                        y=neighborCenters[yCol],
                        z=neighborCenters[zCol],
                        name='%g - %g'%(kClass,nbPoint),
                        mode='lines',
                        line=dict(color='black')
                    )
                )

plotData=clustDat
#plotData['Cluster']=labels
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
#scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=6,
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.25)))
    else:
        kClass,groupData=plotGroup
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=2,
                                 color='black',
                                 opacity=.0625)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675,
            scene = dict(
                    xaxis_title=xCol,
                    yaxis_title=
           yCol,         zaxis_title=zCol)))

fig.show()

In [ ]:
cTest=[202,135,211,186,63]

plotClusters=cTest
xCol='PCAcyl_2' #'X' #
yCol='PCAcyl_3' #'Y' #
zCol='PCAcyl_1' #'Z' #

plotData=clusterCenteroids
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     name='Center_%g'%kClass,
                     mode='markers',
                     marker=dict(size=12,
                                 symbol='circle-open',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.75)))

        

        neighborPoints=neiList[kClass]
        for nbPoint in neighborPoints:
            if nbPoint in plotClusters:
                neighborCenters=plotData[plotData['Cluster'].isin([kClass,nbPoint])]
                scatterPlots.append(
                    go.Scatter3d(
                        x=neighborCenters[xCol],
                        y=neighborCenters[yCol],
                        z=neighborCenters[zCol],
                        name='%g - %g'%(kClass,nbPoint),
                        mode='lines',
                        line=dict(color='black')
                    )
                )

plotData=clustDat
#plotData['Cluster']=labels
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
#scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=6,
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.25)))
    else:
        kClass,groupData=plotGroup
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=2,
                                 color='black',
                                 opacity=.0625)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675,
            scene = dict(
                    xaxis_title=xCol,
                    yaxis_title=
           yCol,         zaxis_title=zCol)))

fig.show()

## Define restraint LCOD coefficients and restraint interval boundaries

Each cell will need to have a set of restraints defined that will restrict the motion of our ion to within that cell. These will be defined by constructing linear combinations of our distance vectors which will be restrained to be within some given interval via a flat bottom harmonic well. Such restraints effectively define the half plane restraints we require.

To employ these restraints to a give cell, $i$, we need to calculate a half plane restraint for each connected cell $j$.

Each half plane is defined by a vector, containing the $\kappa_{ijD}$ entries and a scalar describing the upper limit of the half plane along the LCOD vector described by $\vec{\kappa_{ij}}$

Therefore, the set of all needed restraints for cell $i$ can be described by an nRestraints by nDistances matrix / array, $K_i$, describing the LCOD coefficients of each halfplane, and an nRestraints length vector describing the corresponding upper limits of each LCOD variable.

Using this above description, we can then iterate over all of our defined clusters to ensure that our setup assigns each point from our ion coordinate set to the appropriate cluster. This is an important validation because we originally constructed our clusters based on XYZ coordinates, but we seek to restrict them based on linear combinations of distances measured from our 5 control points. Those corresponding LCOD vectors will not describe our 3D space in the same way as our cartesian coordinates, so we need to be careful to ensure that our cell descriptions under the LCOD basis describe the same set of points as the clusters they are seeking to emulate. In fact, they are expected to be imperfect due to the non-linear nature of distance based restraints, what is important, then, is that they are sufficiently similar that we can use them to describe our milestoning simulation properly. Visualization will be quite useful here.

Most importantly, since the triangulation / tessaltion being used to define our cells is based upon the notion that each cell is defined as points which are closer that cell's control point than any other cells, we could also calculate cell occupancy by computing the distance of a given ion to each cell control point and assigning it into the cell which yields the minimum distance. Indeed, this will be the general method of indexing used during milestoning analysis.

Therefore, a final and very important check is to ensure that we get identical results by using either our LCOD interval or PCA distance based assignment methods. Any discrepancy in assignments between those two methods would indicate a bug or error that must be addressed.

Once we are satisfied the restarints are working we will need to then create functions that can write out corresponding AMBER restraint files (or corresponding restraint files for whatever simulation engine we are using). Due to large number of half plane restraints that each cell may need, trying to write these by hand would be exceeding cumbersome, tedious, and prone to error, so an automated approach is highly desireable here.

First, lets load our ion and cluster data for testing

### Load Data

In [3]:
dataDir='simulationData'
pcaComponents=np.loadtxt('/'.join([dataDir,'pca_components.cylindricalFilter.txt']))
display(pcaComponents)
pcaCylData=pd.read_csv('/'.join([dataDir,'pca_projection_data.cylindricalFilter.csv']))
clustDat=pd.read_csv('/'.join([dataDir,'cluster_data.cylindricalFilter.csv']))
display(clustDat.head())

clusterCenteroids=pd.read_csv('/'.join([dataDir,'Agglomerative_clustering_centers.csv']))
clustDatGroups=clustDat.groupby('Cluster')

clustVols=[]
for groupName,groupData in clustDatGroups:
    cHull=sp.spatial.ConvexHull(groupData[['X','Y','Z']])
    clustVols.append((groupName,cHull.volume))
    

clustVol=pd.DataFrame(clustVols,columns=['Cluster','Volume'])
display(clustVol.head()) #[clustVol['Cluster'].isin(cTest)])

clusterCenteroids=clusterCenteroids.set_index('Cluster').join(
    clustVol.set_index('Cluster'),rsuffix='').reset_index()
display(clusterCenteroids) #[clusterCenteroids['Cluster'].isin(cTest)])


clusterSummary=clusterCenteroids
tessellationClusters=clusterSummary.index.to_numpy() #[0,1,2,3,4,8,10] 
centeroidColumns=np.concatenate([
    ['Cluster'],
    clusterSummary.columns[3:6],
    clusterSummary.columns[11:]])
tessellationCoordinateColumns=['PCAcyl_2','PCAcyl_3','PCAcyl_1'] #[colName for colName in clusterCenteroids.columns \
                               #if 'PCAcyl' in colName]

tessellationData=clusterCenteroids[
        clusterCenteroids['Cluster'].isin(tessellationClusters)
    ][tessellationCoordinateColumns]
clusterVoronoi=sp.spatial.Voronoi(tessellationData)
clusterDelaunay=sp.spatial.Delaunay(tessellationData)
print('cluster centeroids')
display(clusterCenteroids)
print('cluster vertices')
print(clusterDelaunay.vertices)
print()
print("cluster adjacency")
neiList=defaultdict(set)
for p in clusterDelaunay.vertices:
    for i,j in itertools.combinations(p,2):
        neiList[tessellationClusters[i]].add(tessellationClusters[j])
        neiList[tessellationClusters[j]].add(tessellationClusters[i])

for key in sorted(neiList.keys()):
    print("%d:%s" % (key,','.join([str(i) for i in neiList[key]])))

array([[ 0.40045854,  0.08449305,  0.65258185,  0.46235539,  0.43915636],
       [-0.14434161,  0.90046587, -0.30072427,  0.20338675,  0.19111636],
       [ 0.27838328,  0.07815429, -0.03906275, -0.75636341,  0.58547685],
       [ 0.82572275,  0.19911421, -0.15603285, -0.05535959, -0.50112327],
       [ 0.24389168, -0.3691444 , -0.67663114,  0.41195998,  0.42036715]])

,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_1178,D_1366,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster
0,5,1,165552,POT,109.3891,81.3629,25.0487,90.4509,136.76246,65.44026,...,128.234104,47.483162,74.414691,95.227241,3.949924,58.126858,10.976897,-16.407481,0.257267,130
1,6,1,165554,POT,42.9653,78.8911,51.0144,90.4509,136.76246,65.44026,...,112.140092,72.832906,39.343976,110.823964,25.844157,26.064522,55.562696,3.708095,-9.054124,27
2,7,1,165557,POT,59.0333,65.9134,93.0642,90.4509,136.76246,65.44026,...,99.317973,92.829412,60.282709,120.622686,51.407127,14.776168,43.409862,-8.833230,-5.342075,99
3,8,1,165570,POT,74.8840,133.9131,21.6081,90.4509,136.76246,65.44026,...,102.972238,24.265077,47.409354,58.214160,-34.763068,27.159143,13.749322,17.308974,3.062724,245
4,9,1,165581,POT,57.8748,122.7727,107.9835,90.4509,136.76246,65.44026,...,43.343148,84.424088,48.938661,85.049333,19.177857,-41.791883,26.318759,-2.578299,0.682071,1


,Cluster,Volume
0,0,6031.972613
1,1,2770.546167
2,2,2727.414220
3,3,4566.164794
4,4,3836.497178


,Cluster,Frame,AtomID,X,Y,Z,X_Center,Y_Center,Z_Center,R,...,D_1178,D_1366,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Volume
0,0,1469.468750,165962.562500,44.682934,166.358437,128.804662,92.709069,139.608313,62.607678,55.385660,...,37.869426,118.323749,88.049435,98.259369,80.191258,-52.011713,13.472579,14.154026,10.851485,6031.972613
1,1,928.492063,165651.460317,55.305103,122.662275,111.099722,92.833789,139.335496,62.610575,41.299602,...,49.906499,91.289160,55.021504,93.321235,33.509643,-36.156256,28.788391,-0.943710,1.334275,2770.546167
2,2,1030.756098,165566.560976,108.445688,191.335907,111.924205,92.953200,140.221891,62.604540,53.584732,...,41.556516,103.711981,105.311968,60.844022,53.390477,-44.647408,-26.975911,16.142427,5.201749,2727.414220
3,3,883.632653,165721.836735,134.698949,156.352233,8.789557,92.758402,139.430772,62.830846,45.561842,...,121.596911,52.830699,100.762198,56.624222,10.474347,45.507644,-26.464210,16.573817,-1.188909,4566.164794
4,4,1375.270270,165993.000000,95.947468,99.063916,117.130900,92.740730,139.467588,62.735740,40.748084,...,68.521636,92.023583,75.995928,100.809818,39.680890,-10.721619,12.570546,-20.549207,0.353862,3836.497178
5,5,1699.490625,165836.375000,124.502313,150.131436,36.608604,92.668402,139.372557,62.769959,33.749304,...,93.109274,35.751579,78.163873,35.335097,-31.865457,19.566910,-29.644421,6.949978,-2.850368,1019.512611
6,6,602.686441,165834.302260,93.047174,132.199605,95.254139,92.781029,139.123482,62.967223,7.071798,...,41.430811,65.181852,56.051370,61.283966,-12.123558,-36.701096,-0.321793,-12.018503,0.383847,238.993459
7,7,926.197802,165900.752747,70.050105,143.450715,82.157741,92.873919,139.368099,62.677121,23.427888,...,41.603748,60.781755,39.109013,56.319572,-18.955903,-41.793127,13.969030,4.578204,-2.091859,1385.539031
8,8,1654.411765,165893.862745,45.350378,143.519451,11.747867,92.578979,139.389225,62.769648,47.665632,...,112.277517,57.781801,52.904474,79.560198,13.147736,25.848869,30.780136,30.431463,-3.691003,3237.814495
9,9,1092.140000,165974.760000,117.703324,181.618822,133.913760,92.583530,139.195322,62.934877,49.622865,...,45.738775,118.950822,118.175601,82.680211,81.670471,-39.555833,-22.489678,7.984034,9.314392,4332.796567


cluster centeroids


,Cluster,Frame,AtomID,X,Y,Z,X_Center,Y_Center,Z_Center,R,...,D_1178,D_1366,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Volume
0,0,1469.468750,165962.562500,44.682934,166.358437,128.804662,92.709069,139.608313,62.607678,55.385660,...,37.869426,118.323749,88.049435,98.259369,80.191258,-52.011713,13.472579,14.154026,10.851485,6031.972613
1,1,928.492063,165651.460317,55.305103,122.662275,111.099722,92.833789,139.335496,62.610575,41.299602,...,49.906499,91.289160,55.021504,93.321235,33.509643,-36.156256,28.788391,-0.943710,1.334275,2770.546167
2,2,1030.756098,165566.560976,108.445688,191.335907,111.924205,92.953200,140.221891,62.604540,53.584732,...,41.556516,103.711981,105.311968,60.844022,53.390477,-44.647408,-26.975911,16.142427,5.201749,2727.414220
3,3,883.632653,165721.836735,134.698949,156.352233,8.789557,92.758402,139.430772,62.830846,45.561842,...,121.596911,52.830699,100.762198,56.624222,10.474347,45.507644,-26.464210,16.573817,-1.188909,4566.164794
4,4,1375.270270,165993.000000,95.947468,99.063916,117.130900,92.740730,139.467588,62.735740,40.748084,...,68.521636,92.023583,75.995928,100.809818,39.680890,-10.721619,12.570546,-20.549207,0.353862,3836.497178
5,5,1699.490625,165836.375000,124.502313,150.131436,36.608604,92.668402,139.372557,62.769959,33.749304,...,93.109274,35.751579,78.163873,35.335097,-31.865457,19.566910,-29.644421,6.949978,-2.850368,1019.512611
6,6,602.686441,165834.302260,93.047174,132.199605,95.254139,92.781029,139.123482,62.967223,7.071798,...,41.430811,65.181852,56.051370,61.283966,-12.123558,-36.701096,-0.321793,-12.018503,0.383847,238.993459
7,7,926.197802,165900.752747,70.050105,143.450715,82.157741,92.873919,139.368099,62.677121,23.427888,...,41.603748,60.781755,39.109013,56.319572,-18.955903,-41.793127,13.969030,4.578204,-2.091859,1385.539031
8,8,1654.411765,165893.862745,45.350378,143.519451,11.747867,92.578979,139.389225,62.769648,47.665632,...,112.277517,57.781801,52.904474,79.560198,13.147736,25.848869,30.780136,30.431463,-3.691003,3237.814495
9,9,1092.140000,165974.760000,117.703324,181.618822,133.913760,92.583530,139.195322,62.934877,49.622865,...,45.738775,118.950822,118.175601,82.680211,81.670471,-39.555833,-22.489678,7.984034,9.314392,4332.796567


cluster vertices
[[  5 173 138  89]
 [162 252 147 113]
 [162 241 252 113]
 ...
 [165  21  51 155]
 [165  21 214 155]
 [165  21 214  51]]

cluster adjacency
0:32,96,34,167,199,43,237,110,239,121,61,62,127
1:33,69,7,72,233,235,115,85,247,185,251,93,31
2:224,167,40,9,11,109,60,239,92,142,148,188
3:164,136,234,45,78,46,17,23,125,95
4:67,135,202,14,47,240,16,81,83,180,182,186,28,253,191
5:228,138,204,173,17,23,216,89,30,255
6:230,199,7,231,137,235,236,237,109,47,177,114,213,151,59,190
7:1,6,137,151,31,32,34,185,59,61,197,199,233,235,110,247,121,251,127
8:97,162,66,196,198,38,241,50,119,254
9:224,2,197,165,40,137,236,60,207,142,51,155,92,190,127
10:99,100,227,70,105,170,28,240,241,16,83,19,181,249,220,254
11:96,2,40,73,239,92,210,148,188,62,255
12:64,134,231,106,108,140,207,143,179,117,214,215,25,155,189
13:159,37,166,135,204,76,206,111,176,81,212,55,87,126,63
14:67,4,101,104,186,235,47,16,83,180,150,151,182,154,28
15:193,100,70,141,240,145,244,245,217,220,63,126,223
16:194,227,100,4,101,135

# Compute Restraint Information

Recall above:

Our $\kappa_{ijd}$ coefficients are then readily visibile as the terms being summed by the left hand side of the inequallity shown in the last line of equation <b>(4)</b> after swapping the positions of the PCA and distance coordinate summations:

<font size=4>
$$
\begin{align}
    LCOD_{ij} & = \sum\limits_{D \in Distances}{(\sum\limits_{k \in PCAmodes} \alpha_{kD}(R_{ik} - c_{ijk})r_{D}) } \nonumber \\
    & = \sum\limits_{D \in Distances}{\kappa_{ijd}r_D};  \nonumber \\
    where & : \kappa_{ijD} = \sum\limits_{k \in PCAmodes}{\alpha_{kD} (R_{ik} - c_{ijk})} \tag{7}
\end{align}
$$
</font>

and the flat bottom region interval for this LCOD restraint, $LCOD_{ij}$ used to restrain our ion to be closer to cell $i$ than cell $j$ is given as:

<font size=4>
\begin{align}
    LCOD_{ij} > \sum\limits_{k \in PCAmodes}{(c_{ijk} R_{iK} - c_{ijk}^2 + (R_{ik} - c_{ijk})(\sum\limits_{D \in Distances}{ \alpha_{kD}\hat{\lambda_D}}) )} \tag{8} \\
\end{align}
</font>

we are now ready to define functions to compute kappa and interval bounds for our restraints and test them on our ion data

### Define LCOD restraint and interval construction functions

In [4]:
def compute_kappa_ijVec(center_i,center_j,PCA_components):
    cij=(center_i+center_j)/2.
    vij=1.*(center_i-cij) # vij = (Ri - cij)
    return(np.array([np.sum(PCA_components[:,D_index]*vij) \
            for D_index in np.arange(PCA_components.shape[1])]))
    
def compute_restraintBound_ij(center_i,center_j,PCA_components,coordinate_means):
    cij=(center_i+center_j)/2.
    vij=1.*(center_i-cij)
    dSum=np.sum([coordinate_means * component for component in PCA_components],axis=1) #summing alpha*lambdaHat over D
    return(np.sum(cij*center_i - cij**2 + vij*dSum))

def compute_kappaMatrix_i(center_i,neighbor_centers,PCA_components):
    return(np.array([compute_kappa_ijVec(center_i,center_j,PCA_components) \
            for center_j in neighbor_centers]))

def compute_restraintBoundArray_i(center_i,neighbor_centers,PCA_components,coordinate_means):
    return(np.array([compute_restraintBound_ij(center_i,center_j,PCA_components,coordinate_means) \
                     for center_j in neighbor_centers]))

### Test restraint construction functions
Lets test these construction functions using the 5 cluster subset we located previously

In [5]:
cTest=[202,135,211,186,63]

pcaColumns=['PCAcyl_2','PCAcyl_3','PCAcyl_1']
pcaInds=[2,3,1]
distColumns=[colName for colName in clusterCenteroids \
             if 'D_' in colName]
dist_means=clustDat[distColumns].agg('mean').to_frame().T[distColumns].to_numpy().flatten()
display('pca distance means:',dist_means)

print(compute_kappa_ijVec(
                  clusterCenteroids.query('Cluster == 135')[pcaColumns].to_numpy(),
                  clusterCenteroids.query('Cluster == 63')[pcaColumns].to_numpy(),
                  pcaComponents[pcaInds]))
print(compute_restraintBound_ij(
                  clusterCenteroids.query('Cluster == 135')[pcaColumns].to_numpy(),
                  clusterCenteroids.query('Cluster == 63')[pcaColumns].to_numpy(),
                  pcaComponents[pcaInds],
                  dist_means))

ci=135
cj_list=list(neiList[ci])
print(compute_kappaMatrix_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds]))
print(compute_restraintBoundArray_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds],
    dist_means))

'pca distance means:'

array([63.90341628, 78.06315127, 60.4284268 , 68.05441783, 67.62657728])

[ 1.03825516 -6.74152917  2.25626825 -1.53502757 -1.38637915]
-528.3274483306336
[[  4.93612673   3.05883282  -1.6988086    4.76398185  -7.58091081]
 [  8.7490963   -3.82728277   0.51180759  -5.34284141  -2.37723144]
 [  1.20730492  -1.0563638    0.03474756   8.5527949   -9.9539184 ]
 [  1.03825516  -6.74152917   2.25626825  -1.53502757  -1.38637915]
 [  6.20930017  -1.51688157   0.05458457  -8.3029378    3.29013984]
 [  6.82134839   2.95482329  -1.73170858  -0.89505203  -3.27312914]
 [ -9.90361625  -9.86392392   4.44661823   1.09528261   3.16795704]
 [ -2.78277931 -10.01643911   3.7272164    1.48417619  -2.63647194]
 [ -5.33676961  -9.82231743   3.92839373   1.71158184  -0.88325044]
 [ -2.04139528   3.6921213   -1.12450571   2.82265966  -0.14961485]
 [ -7.79650554  -8.42692062   3.69075992   2.50339773   0.61074154]
 [  1.44190277   3.62662382  -1.3887897   -0.84569741   0.94149821]
 [ -2.08309594   6.46828704  -2.02248972   0.3733385    3.26738581]
 [  7.90448241  -3.5396322    0.428

In [6]:
def check_occupancy(rD,kappaMatrix,upperBounds,verbose=False):
    LCODvec=np.array(np.matrix(kappaMatrix)*(np.matrix(rD).T)).flatten()
    if verbose:
        print('rD:',rD)
        print('LCODvec:',LCODvec)
        print('bounds:',upperBounds)
        print('bound checks:',LCODvec>=upperBounds)
    return(np.product(LCODvec >= upperBounds))

In [7]:
ci=135
cj_list=list(neiList[ci])

distColumns=[colName for colName in clusterCenteroids \
             if 'D_' in colName]
dist_means=clustDat[distColumns].agg('mean').to_frame().T[distColumns].to_numpy().flatten()
display('pca distance means:',dist_means)


kappaMat=compute_kappaMatrix_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds])
iBounds=compute_restraintBoundArray_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds],
    dist_means)

coordA=clustDat.query('Cluster == %g'%ci)[distColumns].head(n=1).to_numpy()
coordB=clustDat.query('Cluster == %g'%cj_list[0])[distColumns].head(n=1).to_numpy()

print(check_occupancy(coordA,kappaMat,iBounds,verbose=True))
print(check_occupancy(coordB,kappaMat,iBounds,verbose=True))

'pca distance means:'

array([63.90341628, 78.06315127, 60.4284268 , 68.05441783, 67.62657728])

rD: [[40.31720121 65.88678606 30.9950432  42.44454699 43.06012535]]
LCODvec: [ 223.66291395 -212.70315355  -85.44572749 -457.23622554  -58.6495956
  237.09567408 -728.46380822 -707.15124676 -705.94936633  239.46863534
 -622.60654543  258.68026255  436.04311614 -203.40622418  401.80669701
 -492.36840308 -278.15142632  165.9433543 ]
bounds: [ 103.30810788 -345.5498614  -277.67061728 -528.32744833 -174.03048967
  171.32757623 -974.34929694 -863.87481183 -873.71088566  218.56918875
 -794.3477688   249.37967126  392.1033897  -318.03852373  333.0375021
 -596.02420649 -439.74761013   46.74056115]
bound checks: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
1
rD: [[54.52214442 84.01823264 18.1767028  38.53339065 48.76295751]]
LCODvec: [  309.15194477  -157.0376982   -178.11168344  -595.54571528
    52.58742316   394.59997367 -1091.20708465  -996.90993012
 -1021.93748055   279.935437    -939.76536536   371.39718568
   566.83038696 

### Test on cluster 135
Lets check to see how well the occupancy function can correctly assign points into cluster 135
i.e. we want to know when it puts a point from another cluster into cluster 135 (false positives) 
and what points in cluster 135 it fails to correctly assign to cluster 135 (false negatives).

In [8]:
tqdm.tqdm.pandas(tqdm.tqdm_notebook)

In [9]:
ci=135
cj_list=list(neiList[ci])

distColumns=[colName for colName in clusterCenteroids \
             if 'D_' in colName]
dist_means=clustDat[distColumns].agg('mean').to_frame().T[distColumns].to_numpy().flatten()
display('pca distance means:',dist_means)


kappaMat=compute_kappaMatrix_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds])
iBounds=compute_restraintBoundArray_i(
    clusterCenteroids.query('Cluster == %s'%ci)[pcaColumns].to_numpy(),
    clusterCenteroids.query('Cluster in %s'%cj_list)[pcaColumns].to_numpy(),
    pcaComponents[pcaInds],
    dist_means)

checkDat=clustDat.copy()
checkDat['occupancy_135']=checkDat[distColumns].progress_apply(
        lambda x: check_occupancy(x.to_numpy(),kappaMat,iBounds),
        axis=1
    )
checkDat['Check_135']=checkDat[['Cluster','occupancy_135']].apply(
    lambda x: (x[0]==135)==x[1],axis=1)
checkDat.query('Check_135 == False').groupby('Cluster').count()

'pca distance means:'

array([63.90341628, 78.06315127, 60.4284268 , 68.05441783, 67.62657728])

100%|██████████| 17109/17109 [00:00<00:00, 19882.05it/s]


,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_1366,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,occupancy_135,Check_135
Cluster,,,,,,,,,,,,,,,,,,,,,
63,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
135,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
186,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
202,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


### Analyze false positive / negative rates
In the cell above we displayed counts of all the incorrectly assigned points by cluster... so for cluster 135,
this is the number of false positives, and for the other clusters, these are false negatives.
Lets get a look at the percentage of false negative / false positives by cluster.

We again note that such discrepancies are expected since we know that our LCOD intervals, which are based upon radial distance measurements, do not map linearly into XYZ cartesian coordinates. Thus, we expect that even though our cells should be convex and exhbit planar boundaries in LCOD / PCA space, the same corresponding projections back to XYZ space will yield cells with curved boundaries. This necessarily means that most cells will be concave on at least one boundary. That also means that some points get assigned to different LCOD cells than XYZ cluster from which that cell was constructed.

In [10]:
display(checkDat.query('Check_135 == False').groupby('Cluster').count()['Check_135'])
display(100* checkDat.query('Check_135 == False').groupby('Cluster').count()['Check_135'] / \
        checkDat.groupby('Cluster').count().iloc[[63,135,186,202]]['Check_135'])

Cluster
63      6
135    12
186     8
202     6
Name: Check_135, dtype: int64

Cluster
63     5.940594
135    7.547170
186    5.031447
202    5.000000
Name: Check_135, dtype: float64

### Cross Check PCA-Distance Assignment vs LCOD-Interval Assignment
As we mentioned earlier, our LCOD interval based definition of cell assignment should match exactly with assignment based on distance (in PCA space) to the nearest cell center.

Under our LCOD method, cell assignment must be performed over each cell individually. To that end, we first construct a dictionary of LCOD cells.

### Construct LCOD interval dictionary

In [11]:
pcaColumns=['PCAcyl_2','PCAcyl_3','PCAcyl_1']
pcaInds=[2,3,1]
componentData=pcaComponents[pcaInds]

distColumns=[colName for colName in clusterCenteroids \
             if 'D_' in colName]
dist_means=clustDat[distColumns].agg('mean').to_frame().T[distColumns].to_numpy().flatten()

lcod_dict={}
for iCluster in tqdm.tqdm_notebook(clusterCenteroids.Cluster.unique(),desc="LCOD Generation"):
    iCluster_center=clusterCenteroids.query('Cluster == %g'%iCluster)[pcaColumns].to_numpy()
    jCluster_list=list(neiList[iCluster])
    jCluster_centers=clusterCenteroids.query('Cluster in %s'%jCluster_list)[pcaColumns].to_numpy()
    lcod_dict[iCluster]=(compute_kappaMatrix_i(iCluster_center,jCluster_centers,componentData),
                         compute_restraintBoundArray_i(iCluster_center,jCluster_centers,componentData,dist_means))
print("lcod interval data for cell %g:"%list(lcod_dict.keys())[0],
      lcod_dict[list(lcod_dict.keys())[0]])


lcod interval data for cell 0: (array([[ 3.57799009e+00,  1.12372282e+00, -7.84844376e-01,
         5.46178591e-01, -2.88766590e+00],
       [ 2.78894692e+00, -8.55173466e-01,  1.52186840e-02,
        -7.78373759e-01, -1.58177719e+00],
       [-2.95690175e+00, -3.75631217e+00,  1.70251949e+00,
        -3.68333656e+00,  4.76704013e+00],
       [ 1.71392467e-01, -6.47600212e-01,  1.57888550e-01,
         2.02777713e+00, -2.40121022e+00],
       [-3.76830543e+00, -4.22624348e+00,  1.94927532e+00,
        -3.56852326e+00,  5.10980309e+00],
       [-1.74792206e+00, -2.13810142e+00,  1.08086064e+00,
        -6.55275234e+00,  7.29802779e+00],
       [-2.52882956e+00, -3.38106717e+00,  1.43601252e+00,
         1.79233367e-01,  6.33903688e-01],
       [ 1.85388080e+00, -5.11764023e+00,  1.65948902e+00,
        -3.75389372e+00,  7.80321644e-01],
       [ 3.89624006e+00,  2.53249184e-01, -3.76175315e-01,
        -5.47858584e+00,  2.72535843e+00],
       [ 3.86195826e+00,  7.43914775e+00, -2.8721

### Define LCOD based cell assignment
Using our dictionary of LCOD cells, we can now check for cell occupancy of any ion against all defined cells. We can then count how many cells the ion 'occupies'. If this value is anything except a 1, then we have a problem.

In [12]:
def count_lcod_occupancy(rD,LCOD_dict):
    return(np.sum([
        check_occupancy(rD,LCOD_dict[keyVal][0],LCOD_dict[keyVal][1]) \
        for keyVal in LCOD_dict
    ]))

In [13]:
clustDat['LCOD_Count']=clustDat[distColumns].progress_apply(
    lambda x: count_lcod_occupancy(x,lcod_dict),axis=1)
clustDat.query('LCOD_Count != 1')

100%|██████████| 17109/17109 [04:56<00:00, 57.65it/s]


,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_1366,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster,LCOD_Count


### Assign ions to cells using LCOD intervals

In [14]:
def assign_lcod_cell(rD,LCOD_dict):
    cellArr=np.array([keyVal for keyVal in LCOD_dict])
    return(cellArr[np.nonzero([
        check_occupancy(rD,LCOD_dict[keyVal][0],LCOD_dict[keyVal][1]) \
        for keyVal in LCOD_dict
    ])][0])

In [15]:
clustDat['LCOD_Cell']=clustDat[distColumns].progress_apply(
    lambda x: assign_lcod_cell(x,lcod_dict),axis=1)
clustDat.head()

100%|██████████| 17109/17109 [04:56<00:00, 57.71it/s]


,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_2377,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster,LCOD_Count,LCOD_Cell
0,5,1,165552,POT,109.3891,81.3629,25.0487,90.4509,136.76246,65.44026,...,74.414691,95.227241,3.949924,58.126858,10.976897,-16.407481,0.257267,130,1,130
1,6,1,165554,POT,42.9653,78.8911,51.0144,90.4509,136.76246,65.44026,...,39.343976,110.823964,25.844157,26.064522,55.562696,3.708095,-9.054124,27,1,27
2,7,1,165557,POT,59.0333,65.9134,93.0642,90.4509,136.76246,65.44026,...,60.282709,120.622686,51.407127,14.776168,43.409862,-8.833230,-5.342075,99,1,181
3,8,1,165570,POT,74.8840,133.9131,21.6081,90.4509,136.76246,65.44026,...,47.409354,58.214160,-34.763068,27.159143,13.749322,17.308974,3.062724,245,1,245
4,9,1,165581,POT,57.8748,122.7727,107.9835,90.4509,136.76246,65.44026,...,48.938661,85.049333,19.177857,-41.791883,26.318759,-2.578299,0.682071,1,1,1


In [16]:
def assign_PCAdist_cell(rk,centeroids_PCA,centeroid_labels):
    return(centeroid_labels[
        np.argmin(np.sum(
            (np.atleast_2d(centeroids_PCA)-np.atleast_2d(rk))**2,
            axis=1
        ))])

In [17]:
centeroidDistances=clusterCenteroids[pcaColumns].to_numpy()
centeroidClusters=clusterCenteroids['Cluster'].to_numpy()
clustDat['PCAdist_Cell']=clustDat[pcaColumns].apply(
    lambda x: assign_PCAdist_cell(x,centeroidDistances,centeroidClusters),
    axis=1)
display(clustDat.head())
display(clustDat.query('LCOD_Cell != PCAdist_Cell'))

,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster,LCOD_Count,LCOD_Cell,PCAdist_Cell
0,5,1,165552,POT,109.3891,81.3629,25.0487,90.4509,136.76246,65.44026,...,95.227241,3.949924,58.126858,10.976897,-16.407481,0.257267,130,1,130,130
1,6,1,165554,POT,42.9653,78.8911,51.0144,90.4509,136.76246,65.44026,...,110.823964,25.844157,26.064522,55.562696,3.708095,-9.054124,27,1,27,27
2,7,1,165557,POT,59.0333,65.9134,93.0642,90.4509,136.76246,65.44026,...,120.622686,51.407127,14.776168,43.409862,-8.833230,-5.342075,99,1,181,181
3,8,1,165570,POT,74.8840,133.9131,21.6081,90.4509,136.76246,65.44026,...,58.214160,-34.763068,27.159143,13.749322,17.308974,3.062724,245,1,245,245
4,9,1,165581,POT,57.8748,122.7727,107.9835,90.4509,136.76246,65.44026,...,85.049333,19.177857,-41.791883,26.318759,-2.578299,0.682071,1,1,1,1


,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,D_3796,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster,LCOD_Count,LCOD_Cell,PCAdist_Cell


### Highlight differences between LCOD interval cells vs XYZ clusters

Now that we have assigned each ion to a cell using LCOD (and verified that it is faithfully reproducing assignment based
upon a voronoi tesselation created from the PCA space cluster centers). We can visualize which ions are getting put into
different LCOD cells than the corresponding XYZ cluster

In [18]:
clustDat['Cell_Check']=clustDat['LCOD_Cell']==clustDat['Cluster']
clustDat.head()

,Unnamed: 0,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,...,PCAcyl_0,PCAcyl_1,PCAcyl_2,PCAcyl_3,PCAcyl_4,Cluster,LCOD_Count,LCOD_Cell,PCAdist_Cell,Cell_Check
0,5,1,165552,POT,109.3891,81.3629,25.0487,90.4509,136.76246,65.44026,...,3.949924,58.126858,10.976897,-16.407481,0.257267,130,1,130,130,True
1,6,1,165554,POT,42.9653,78.8911,51.0144,90.4509,136.76246,65.44026,...,25.844157,26.064522,55.562696,3.708095,-9.054124,27,1,27,27,True
2,7,1,165557,POT,59.0333,65.9134,93.0642,90.4509,136.76246,65.44026,...,51.407127,14.776168,43.409862,-8.833230,-5.342075,99,1,181,181,False
3,8,1,165570,POT,74.8840,133.9131,21.6081,90.4509,136.76246,65.44026,...,-34.763068,27.159143,13.749322,17.308974,3.062724,245,1,245,245,True
4,9,1,165581,POT,57.8748,122.7727,107.9835,90.4509,136.76246,65.44026,...,19.177857,-41.791883,26.318759,-2.578299,0.682071,1,1,1,1,True


In [19]:
cTest=[202,135,211,186,63]

plotClusters=cTest
xCol='X' #'PCAcyl_2' #
yCol='Y' #'PCAcyl_3' #
zCol='Z' #'PCAcyl_1' #

labels=clustDat.Cluster
plotData=clusterCenteroids
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     name='Center_%g'%kClass,
                     mode='markers',
                     marker=dict(size=12,
                                 symbol='diamond-open',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.95)))

        

        neighborPoints=neiList[kClass]
        for nbPoint in neighborPoints:
            if nbPoint in plotClusters:
                neighborCenters=plotData[plotData['Cluster'].isin([kClass,nbPoint])]
                scatterPlots.append(
                    go.Scatter3d(
                        x=neighborCenters[xCol],
                        y=neighborCenters[yCol],
                        z=neighborCenters[zCol],
                        name='%g - %g'%(kClass,nbPoint),
                        mode='lines',
                        line=dict(color='black')
                    )
                )

plotData=clustDat
#plotData['Cluster']=labels
nLabels=np.max(labels)+2
colorpalette=sns.palettes.color_palette('colorblind',nLabels)
go=ply.graph_objs
#scatterPlots=[]
for iGroup,plotGroup in enumerate(plotData.groupby('Cluster')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=6,
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.25)))
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData.query('Cell_Check == False')[xCol],
                     y=groupData.query('Cell_Check == False')[yCol],
                     z=groupData.query('Cell_Check == False')[zCol],
                     mode='markers',
                     name='Mismatch',
                     marker=dict(size=7,
                                 symbol='x',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.5)))
        
    else:
        kClass,groupData=plotGroup
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData[xCol],
                     y=groupData[yCol],
                     z=groupData[zCol],
                     mode='markers',
                     name='Cluster_%g'%kClass,
                     marker=dict(size=2,
                                 color='black',
                                 opacity=.0625)))
        
for iGroup,plotGroup in enumerate(plotData.groupby('LCOD_Cell')):
    kClass,groupData=plotGroup
    if kClass in plotClusters:
        scatterPlots.append(
            go.Scatter3d(
                     x=groupData.query('Cell_Check == False')[xCol],
                     y=groupData.query('Cell_Check == False')[yCol],
                     z=groupData.query('Cell_Check == False')[zCol],
                     mode='markers',
                     name='Cell_%g'%kClass,
                     marker=dict(size=14,
                                 symbol='circle-open',
                                 color=colorpalette.as_hex()[kClass],
                                 opacity=.5)))

fig=go.Figure(
        data=scatterPlots,
        layout=dict(
            width=900,height=675,
            scene = dict(
                    xaxis_title=xCol,
                    yaxis_title=
           yCol,         zaxis_title=zCol)))

fig.show()

In [50]:
amberLCOD_template='''&colvar !{colvarLabel}
   cv_type = 'LCOD'
   cv_i = {atom_pair_list},
   cv_r = {coeff_list},
   anchor_position = {interval_anchors},
   anchor_strength = {interval_strengths}
/

'''

atomIndStr=', '.join(['%s'%distCol.split('_')[-1] for distCol in distColumns])
ci=135
jj=0
cj=list(neiList[ci])[jj]
testCoefs=lcod_dict[ci][0][jj,:]
testCoefStr=', '.join(['%f'%coef for coef in testCoefs])

anchor_locs='-999999, -999999, %f, %f'%(
    lcod_dict[ci][1][jj],lcod_dict[135][1][jj]+5)
anchor_str='0.0, 100.0'

print(amberLCOD_template.format(
    colvarLabel='Cluster Pair: %g - %g'%(ci,cj),
    atom_pair_list=atomIndStr,
    coeff_list=testCoefStr,
    interval_anchors=anchor_locs,
    interval_strengths=anchor_str))

&colvar !Cluster Pair: 135 - 194
   cv_type = 'LCOD'
   cv_i = 958, 1178, 1366, 2377, 3796,
   cv_r = 4.936127, 3.058833, -1.698809, 4.763982, -7.580911,
   anchor_position = -999999, -999999, 103.308108, 108.308108,
   anchor_strength = 0.0, 100.0
/




In [21]:
ionInd=5000
lcodCell=135

lcod_mat=lcod_dict[lcodCell][0]
lcod_rest=lcod_dict[lcodCell][1]

#this needs to get updated with the atom indices themselves
atomIndStr=', '.join(['%s, %g'%(distCol.split('_')[-1],ionInd) for distCol in distColumns])

restStrList=['''cv_file cell %g collective variable restraints'''%lcodCell]
for iRest,restMax in enumerate(lcod_rest):
    testCoefs=lcod_mat[iRest,:]
    testCoefStr=', '.join(['%f'%coef for coef in testCoefs])

    anchor_locs='%f, %f, 999999, 999999'%(
        restMax-5,restMax)
    anchor_str='0.0, 100.0'
    restStrList.append(amberLCOD_template.format(
        atom_pair_list=atomIndStr,
        coeff_list=testCoefStr,
        interval_anchors=anchor_locs,
        interval_strengths=anchor_str))
    
print('\n'.join(restStrList))

cv_file cell 135 collective variable restraints
&colvar
   cv_type = 'LCOD'
   cv_i = 958, 5000, 1178, 5000, 1366, 5000, 2377, 5000, 3796, 5000,
   cv_r = 4.936127, 3.058833, -1.698809, 4.763982, -7.580911,
   anchor_position = 98.308108, 103.308108, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar
   cv_type = 'LCOD'
   cv_i = 958, 5000, 1178, 5000, 1366, 5000, 2377, 5000, 3796, 5000,
   cv_r = 8.749096, -3.827283, 0.511808, -5.342841, -2.377231,
   anchor_position = -350.549861, -345.549861, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar
   cv_type = 'LCOD'
   cv_i = 958, 5000, 1178, 5000, 1366, 5000, 2377, 5000, 3796, 5000,
   cv_r = 1.207305, -1.056364, 0.034748, 8.552795, -9.953918,
   anchor_position = -282.670617, -277.670617, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar
   cv_type = 'LCOD'
   cv_i = 958, 5000, 1178, 5000, 1366, 5000, 2377, 5000, 3796, 5000,
   cv_r = 1.038255, -6.741529, 2.256268, -1.535028, -1.386379,
   anchor_position = -5

In [22]:
CAatomInfoFile='/'.join([dataDir,'CA_atominfo.txt'])
POTatomInfoFile='/'.join([dataDir,'POT_atominfo.txt'])

CAatomInfo=pd.read_csv(CAatomInfoFile,delim_whitespace=True)
display(CAatomInfo)
POTatomInfo=pd.read_csv(POTatomInfoFile,delim_whitespace=True)
display(POTatomInfo)

,Atom,Name,Res,Name.1,Mol,Type,Charge,Mass,GBradius,El,rVDW,eVDW
0,15942,CA,958,LYS,4,CT1,0.07,12.011,1.7,C,2.0,0.032
1,19546,CA,1178,GLN,4,CT1,0.07,12.011,1.7,C,2.0,0.032
2,22493,CA,1366,PHE,4,CT1,0.07,12.011,1.7,C,2.0,0.032
3,39396,CA,2377,CYS,8,CT1,0.07,12.011,1.7,C,2.0,0.032
4,62839,CA,3796,ILE,12,CT1,0.07,12.011,1.7,C,2.0,0.032


,Atom,Name,Res,Name.1,Mol,Type,Charge,Mass,GBradius,El,rVDW,eVDW
0,699118,POT,165503,POT,161261,POT,1.0,39.0983,1.5,K,1.7637,0.087
1,699145,POT,165530,POT,161288,POT,1.0,39.0983,1.5,K,1.7637,0.087
2,699167,POT,165552,POT,161310,POT,1.0,39.0983,1.5,K,1.7637,0.087
3,699169,POT,165554,POT,161312,POT,1.0,39.0983,1.5,K,1.7637,0.087
4,699172,POT,165557,POT,161315,POT,1.0,39.0983,1.5,K,1.7637,0.087
5,699185,POT,165570,POT,161328,POT,1.0,39.0983,1.5,K,1.7637,0.087
6,699196,POT,165581,POT,161339,POT,1.0,39.0983,1.5,K,1.7637,0.087
7,699238,POT,165623,POT,161381,POT,1.0,39.0983,1.5,K,1.7637,0.087
8,699242,POT,165627,POT,161385,POT,1.0,39.0983,1.5,K,1.7637,0.087
9,699326,POT,165711,POT,161469,POT,1.0,39.0983,1.5,K,1.7637,0.087


In [23]:
CAresToindex={x[1]:x[0] for x in CAatomInfo[['Atom','Res']].to_numpy()}
CAresToindex

{958: 15942, 1178: 19546, 1366: 22493, 2377: 39396, 3796: 62839}

In [26]:
clustGroups=clustDat.groupby('Cluster')

nearestIonDict={}
frameDict={}
for clustID,clustGroupDat in tqdm.tqdm_notebook(clustGroups,desc="Computing Nearest Ions"):
    atomIDs=clustGroupDat['AtomID'].to_numpy()
    frames=clustGroupDat['Frame'].to_numpy()
    clustCenter=clusterCenteroids.query('Cluster == %g'%clustID)[pcaColumns].to_numpy()
    centerData=clustGroupDat[pcaColumns].to_numpy()
    distances=np.sqrt(np.sum((centerData-clustCenter)**2,axis=1))
    nearestIonDict[clustID]=atomIDs[np.argmin(distances)]
    frameDict[clustID]=frames[np.argmin(distances)]
    
display(nearestIonDict)
display(frameDict)

{0: 166136,
 1: 165623,
 2: 165554,
 3: 165554,
 4: 166136,
 5: 166052,
 6: 165891,
 7: 165711,
 8: 165872,
 9: 166052,
 10: 165774,
 11: 165711,
 12: 166136,
 13: 166095,
 14: 166095,
 15: 165557,
 16: 165774,
 17: 165891,
 18: 165581,
 19: 165891,
 20: 165891,
 21: 165552,
 22: 165872,
 23: 165554,
 24: 165554,
 25: 166052,
 26: 166052,
 27: 165530,
 28: 165570,
 29: 165872,
 30: 166052,
 31: 165557,
 32: 165872,
 33: 165530,
 34: 165774,
 35: 165530,
 36: 166069,
 37: 166136,
 38: 165581,
 39: 165530,
 40: 166069,
 41: 166136,
 42: 165774,
 43: 166136,
 44: 165774,
 45: 165552,
 46: 165891,
 47: 165530,
 48: 165570,
 49: 166095,
 50: 166095,
 51: 165552,
 52: 166136,
 53: 165711,
 54: 165872,
 55: 165530,
 56: 165552,
 57: 166136,
 58: 166069,
 59: 165872,
 60: 165872,
 61: 165774,
 62: 166136,
 63: 165627,
 64: 166136,
 65: 165570,
 66: 165581,
 67: 165872,
 68: 165627,
 69: 166136,
 70: 165570,
 71: 165774,
 72: 166136,
 73: 166069,
 74: 166069,
 75: 165774,
 76: 165554,
 77: 1656

{0: 1179,
 1: 716,
 2: 995,
 3: 826,
 4: 1864,
 5: 1868,
 6: 554,
 7: 1486,
 8: 1759,
 9: 42,
 10: 980,
 11: 1352,
 12: 1645,
 13: 1728,
 14: 1998,
 15: 395,
 16: 1002,
 17: 1354,
 18: 1978,
 19: 813,
 20: 1114,
 21: 784,
 22: 1183,
 23: 816,
 24: 957,
 25: 55,
 26: 211,
 27: 736,
 28: 734,
 29: 1665,
 30: 1581,
 31: 243,
 32: 1959,
 33: 1317,
 34: 96,
 35: 827,
 36: 191,
 37: 1837,
 38: 1940,
 39: 1646,
 40: 1931,
 41: 461,
 42: 990,
 43: 1993,
 44: 1991,
 45: 1595,
 46: 1361,
 47: 1714,
 48: 145,
 49: 393,
 50: 213,
 51: 879,
 52: 1353,
 53: 420,
 54: 1226,
 55: 1746,
 56: 1906,
 57: 1819,
 58: 1802,
 59: 1049,
 60: 759,
 61: 35,
 62: 1239,
 63: 902,
 64: 1669,
 65: 441,
 66: 1936,
 67: 616,
 68: 1313,
 69: 1892,
 70: 982,
 71: 1629,
 72: 1939,
 73: 1952,
 74: 1571,
 75: 401,
 76: 482,
 77: 670,
 78: 1874,
 79: 1729,
 80: 609,
 81: 755,
 82: 963,
 83: 778,
 84: 208,
 85: 1500,
 86: 236,
 87: 717,
 88: 1292,
 89: 566,
 90: 111,
 91: 1583,
 92: 2040,
 93: 109,
 94: 701,
 95: 788,
 96: 

In [31]:
CAresToindex[int('958')]

15942

In [53]:
outDir='simulationSetupFiles'
restraintFileDict={}

amberDist_template='''&colvar !{colvarLabel}
   cv_type = 'DISTANCE',
   cv_i = {atomPair},
   anchor_postion = {anchor_locs},
   anchor_strength = {anchor_strengths}
/

'''

for cluster in clusterCenteroids.Cluster.unique():
    lcodData=lcod_dict[cluster]
    frameNum=frameDict[cluster]
    
    ionInd=nearestIonDict[cluster]
    lcodCell=cluster

    lcod_mat=lcod_dict[lcodCell][0]
    lcod_rest=lcod_dict[lcodCell][1]

    #this needs to get updated with the atom indices themselves
    atomIndStr=', '.join(
        ['%g, %g'%(
            CAresToindex[int(distCol.split('_')[-1])],
            ionInd) \
         for distCol in distColumns])

    restStrList=['''cv_file cell %g collective variable restraints\n'''%lcodCell]
    for iRest,restMax in enumerate(lcod_rest):
        testCoefs=lcod_mat[iRest,:]
        testCoefStr=', '.join(['%f'%coef for coef in testCoefs])

        anchor_locs='%f, %f, 999999, 999999'%(
            restMax-5,restMax)
        anchor_str='0.0, 100.0'
        ci=cluster
        cj=list(neiList[cluster])[iRest]
        restStrList.append(amberLCOD_template.format(
            colvarLabel='Cluster Pair: %g - %g'%(ci,cj),
            atom_pair_list=atomIndStr,
            coeff_list=testCoefStr,
            interval_anchors=anchor_locs,
            interval_strengths=anchor_str))
    for controlRes in [int(distCol.split('_')[-1]) for distCol in distColumns]:
        restStrList.append(amberDist_template.format(
            atomPair='%g, %g'%(CAresToindex[controlRes],ionInd),
            colvarLabel='Monitoring for: D_%g (atoms %g - %g)'%(
                controlRes,CAresToindex[controlRes],ionInd),
            anchor_locs='0, 1, 2, 3',
            anchor_strengths='0, 0'))
        
    restraintFileDict[cluster]=''.join(restStrList)
gc.collect()
print(restraintFileDict[135])

cv_file cell 135 collective variable restraints
&colvar !Cluster Pair: 135 - 194
   cv_type = 'LCOD'
   cv_i = 15942, 165774, 19546, 165774, 22493, 165774, 39396, 165774, 62839, 165774,
   cv_r = 4.936127, 3.058833, -1.698809, 4.763982, -7.580911,
   anchor_position = 98.308108, 103.308108, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar !Cluster Pair: 135 - 223
   cv_type = 'LCOD'
   cv_i = 15942, 165774, 19546, 165774, 22493, 165774, 39396, 165774, 62839, 165774,
   cv_r = 8.749096, -3.827283, 0.511808, -5.342841, -2.377231,
   anchor_position = -350.549861, -345.549861, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar !Cluster Pair: 135 - 100
   cv_type = 'LCOD'
   cv_i = 15942, 165774, 19546, 165774, 22493, 165774, 39396, 165774, 62839, 165774,
   cv_r = 1.207305, -1.056364, 0.034748, 8.552795, -9.953918,
   anchor_position = -282.670617, -277.670617, 999999, 999999,
   anchor_strength = 0.0, 100.0
/

&colvar !Cluster Pair: 135 - 4
   cv_type = 'LCOD'
   cv_i =

In [54]:
outDir='simulationSetupFiles/'
for cluster in restraintFileDict:
    outFileName='cluster_{cluster}.cv.in'.format(cluster=cluster)
    outFile=open('/'.join([outDir,outFileName]),'w')
    outFile.write(restraintFileDict[cluster])
    outFile.close()

In [2]:
?pd.read_csv